# Optimizing with DSPy

Most folks who have heard about DSPy are familiar with the fact that DSPy offers automated optimizations - both for prompts and for finetuning. It is definitely one of the standout features of DSPy, as using **[Optimizers](https://dspy.ai/learn/optimization/optimizers/?h=optimizer)** is an extremely powerful way to further improve the quality of your solution. 

#### Note
As we saw in the previous notebook, DSPy offers many other benefits beyond optimization. We emphasize this point because you shouldn't discount using DSPy if you don't have the training examples needed (we typically recommend 70+ examples) for optimization when you start building your solution. You can still benefit from all the other features of DSPy, and can skip ahead to notebook 03_register_and_deploy. 

**The defining value proposition of DSPy is providing a declarative approach to developing GenAI solutions.**

Additionally, while most customers won't have a curated dataset when they start a project, they will want to log interactions for their production applications, which can then be used to create a training set for the DSPy Optimizers.

### Optimizing DSPy programs in Databricks

With that said, lets look at how you can leverage DSPy's Optimizers in Databricks.

This notebook looks at optimizing the RAG app that we previously built, once again focusing on highlighting the seamless integration with Databricks & Mlflow native features, rather than the results themselves (as this uses an artificially created training set which hasn't been curated well enough to show material improvement due to Optimization).

In [0]:
%pip install -q dspy databricks-agents mlflow databricks-sdk==0.50.0 
dbutils.library.restartPython()

In [0]:
import dspy
import math
import mlflow
import pandas as pd
from mlflow.models import ModelConfig

from dspy.retrieve.databricks_rm import DatabricksRM

from databricks.agents.evals import judges
from databricks.agents.evals import generate_evals_df

In [0]:
config_file = "config.yaml"
model_config = ModelConfig(development_config=config_file)

In [0]:
CATALOG = model_config.get("catalog")
SCHEMA = model_config.get("schema")
VOLUME = model_config.get("volume")

VECTOR_SEARCH_ENDPOINT = model_config.get("vector_search_endpoint")
VECTOR_SEARCH_INDEX = model_config.get("vector_search_index")
index_path = f"{CATALOG}.{SCHEMA}.{VECTOR_SEARCH_INDEX}"

model = model_config.get("chat_endpoint_name")

LM = f"databricks/{model}"

path = f"{CATALOG}/{SCHEMA}/{VOLUME}"

## Optimization breakdown

A typical DSPy Optimizer requires three things:

1. Your DSPy program. This may be a single module (e.g., dspy.Predict) or a complex multi-module program. In our example, this is our RAG module defined above. 
2. A curated dataset as training inputs - the more the better, but you can start with what's feasible to create/access.
3. Your metric - this is a function that evaluates the output of your program, and assigns it a score (higher is better), and is what the Optimizer will optimize for.

We'll start with the same RAG program module we wrote in the previous notebook

In [0]:
class RAG(dspy.Module):
  def __init__(self, for_mosaic_agent=True): 
    # setup mlflow tracing
    mlflow.dspy.autolog()

    # setup flag indicating if the object will be deployed as a Mosaic Agent
    self.for_mosaic_agent = for_mosaic_agent
    self.lm = dspy.LM(LM, cache=False)

    # setup the primary retriever pointing to the chunked documents
    self.retriever = DatabricksRM(
        databricks_index_name=index_path,
        text_column_name="page_content",
        docs_id_column_name="unique_chunk_index",
        columns=["page_content"],
        k=5,
        use_with_databricks_agent_framework=for_mosaic_agent
    )
    
    self.response_generator = dspy.Predict("context, question -> response")

  def forward(self, question):
    if self.for_mosaic_agent:
      # When using a mosaic agent, questions can be of type ChatAgentMessage or (TODO: accept ChatModelMessage) 
      question = question["messages"][-1]["content"]

    context = self.retriever(
        question
    )

    with dspy.context(lm=self.lm):
      response = self.response_generator(context=context, question=question)

      if self.for_mosaic_agent:
        return response.response
      return response

In [0]:
rag = RAG(for_mosaic_agent=False)

### Load Curated Dataset

In [0]:
dataset = spark.read.csv(f"/Volumes/{path}/DSPy Databricks QnA - Sheet1.csv", multiLine=True, header=True, quote='"', escape='"')

In [0]:
dataset.display()

#### DSPy Examples
Once we've loaded our dataset, we need to convert each row to a DSPy **[Example](https://dspy.ai/learn/evaluation/data/)** in order to pass them to the Optimizer. The easiest way to create an Example is to pass in a dictionary. Examples are just dictionaries with some additional useful utilities. 

Use `with_inputs(<input>)` to specify which of the provided fields are input fields. Everything else is considered a label or metadata.

In [0]:
trainset, valset = dataset.randomSplit([0.7, 0.3], seed=15)

trainset = trainset.select("Question", "Answer").rdd.map(
    lambda row: dspy.Example({"question": row["Question"], "response": row["Answer"]}).with_inputs("question")
).collect()

valset = valset.select("Question", "Answer").rdd.map(
    lambda row: dspy.Example({"question": row["Question"], "response": row["Answer"]}).with_inputs("question")
).collect()

Lets inspect one of our examples to ensure it's formatted right

In [0]:
ex = trainset[0]
ex

In [0]:
ex.question

In [0]:
ex.response

### Define Metric

Now that we have our program, and our curated dataset as DSPy Examples, the last step is to define a **[Metric](https://dspy.ai/learn/evaluation/metrics/)**. 

From a programming perspective, Metrics are simple. They take Examples and the output from your program, and return a score which quantifies how good the output is against the reference Example. The challenging part is choosing your Metric. For example, accuracy isn't the best metric for long form answers, as you're unlikely to ever get a word for word matching output, even if the response is 'correct'. 

#### Mosaic Agent Eval Judges

You can define your own Metric, as long as it takes an `Example` and a prediction from your program, and outputs a score. In this notebook we'll use the Mosaic Agent Eval LLM correctness judge to define our Metric.  


In [0]:
def evalute_using_mosaic_agent(example, pred, trace=None):
    # Ref: https://docs.databricks.com/aws/en/generative-ai/agent-evaluation/llm-judge-metrics#call-judges-using-the-python-sdk
    # Running evaluation using the Mosaic Agent Evaluation
    return judges.correctness(
        request=example.question,
        response=pred.response,
        expected_response=example.response,
        ).value.name == "YES"

#### Running Optimizer 

A DSPy optimizer is an algorithm that can tune the parameters of a DSPy program (i.e., the prompts and/or the LM weights) to maximize the metrics you specify. DSPy offers a number of different Optimizers (with new ones coming out frequently!), but they all have the same interface. In this example we'll use MIPRO with our `evalute_using_mosaic_agent` Metric.

We'll save our optimized program as "optimized_rag.json" so that we can load and use it later

In [0]:
from dspy.evaluate.evaluate import Evaluate
from dspy.teleprompt import MIPROv2

# Set up a bootstrap optimizer, which optimizes the RAG program.
optimizer = MIPROv2(
    metric=evalute_using_mosaic_agent, # Use defined evaluation function
    prompt_model=dspy.LM(LM)
)

# Start a new MLflow run to track all evaluation metrics
with mlflow.start_run(run_name="dspy_rag_optimization"):
    # Optimize the program by identifying the best few-shot examples for the prompt used by the `response_generator` step
    optimized_rag = optimizer.compile(rag, 
                                    trainset=trainset,
                                    max_bootstrapped_demos=3,
                                    requires_permission_to_run=False
                                    )

optimized_rag.save("optimized_rag.json")

### Evaluate Optimized Program

You'll see a new file "optimized_rag.json" has been created by running the cell above. You can view the file to see the Optimized prompt that has been created. 

Lets now load the optimized prompt in order to test it.

In [0]:
optimized_rag = RAG(for_mosaic_agent=True)
optimized_rag.load("optimized_rag.json")

In [0]:
result = optimized_rag({'messages': [{'content': 'What features are included in the "Machine learning tutorial" notebook for the scikit-learn package?', 'role': 'user'}]})

#### Under the hood 
As we saw in the previous notebook, the MLFlow trace helps us understand the steps DSPy took under the hood. Since it's the same program, the internal steps are the same.

If you click into `LM.__call__` you'll see that the few shot examples created by MIPROv2 are being passed to the LLM. 

## Evaluation

Just as we evaluated the unoptimized DSPy program against our synthetic evaluation dataset, we can do the same for the optimized program. 

#### Load synthetic eval table

In [0]:
eval_table = model_config.get("eval_table")
eval_table_path = f"{CATALOG}.{SCHEMA}.{eval_table}"
eval_df = spark.table(eval_table_path)

In [0]:
eval_df.display()

#### Run evaluation

We simply pass in the `optimized_rag` module we created and our evaluation dataset to the mlflow `evaluate` call, and we can see the results in the mlflow run under "traces". 

Please note that this is an engineered example and the focus of this notebook is not on the quality of the results (the vector search is quite poor for now!) but to show the dev flow with DSPy and the integration with Databricks native capabilities. 

In [0]:
mlflow.evaluate(
  model=optimized_rag,
  data=eval_df,
  model_type="databricks-agent"
)

## Summary

In this notebook we've seen how:
1. We can optimize a DSPy program using the Mosaic Agent Eval LLM judges as a metric
2. We can save an optimized DSPy program
3. We can evaluate the optimized DSPy program against an evaluation dataset